# Working With Elections Data

In [ ]:
# Inline Chart Parameters
%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

# R imports
%load_ext rpy2.ipython

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

# Python Imports
import pandas as pd
import numpy as np
import us
from utils import download_pres_results
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
%%R

smartRequire <- function(pkg) {
  if (!require(pkg, character.only = T)) { install.packages(pkg, repos='http://cran.us.r-project.org') }
  require(pkg, character.only = T)
}

libs <- c("ggplot2", "readr", "gridExtra")
lapply(libs, smartRequire)
sprintf("Successfully loaded: %s", libs)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

Today's Datasets
- Election Results
    - [Presidential Results Compiled by DailyKos (via U.S. ElectionAtlas)](https://docs.google.com/spreadsheets/d/1D-edaVHTnZNhVU840EPUhz3Cgd7m39Urx7HM8Pq6Pus/edit#gid=29622862)
    - [Presidential Results @ County Level for 2016 - US Election Atlas]()
- [Polls](https://github.com/fivethirtyeight/data/tree/master/pollster-ratings)
- [Pollster Ratings](https://github.com/fivethirtyeight/data/tree/master/pollster-ratings)

## Election Results

### PVI

In [ ]:
pres_results_raw = download_pres_results()

def state_to_abbrev(state):
    return us.states.lookup(state).abbr
pres_results_national = pres_results_raw[pres_results_raw.state == 'Nationwide']
pres_results_national['d_margin'] = pres_results_national.DEM - pres_results_national.REP

pres_results_by_state = pres_results_raw[~pres_results_raw.state.isin(['Washington DC','Nationwide'])]
pres_results_by_state['d_margin'] = pres_results_by_state.DEM - pres_results_by_state.REP

In [ ]:
%%R -i pres_results_national
# pres_results_national
ggplot(data=pres_results_national,aes(x=year, y=d_margin)) + 
geom_bar(stat="identity") +
ggtitle("Presidential Election Results - Popular Vote")

In [ ]:
%%R -i pres_results_by_state

pres_results_by_state
ggplot(data=pres_results_by_state,aes(x=year, y=d_margin, group=state, alpha=.1,col = ifelse(state=='California','gray','red'))) + 
geom_line()

In [ ]:
pres_results_national.head()

In [ ]:
df = pres_results_by_state

df['previous_year'] = df.year - 4

df = pd.merge(
    df, 
    pres_results_by_state[['year','state','REP', 'DEM']],
    how='left', left_on=('previous_year', 'state'), right_on=('year', 'state'), suffixes=('','_previous_year'))

df = pd.merge(
    df, 
    pres_results_national[['year','REP', 'DEM']],
    how='left', on='year', suffixes=('','_national'))

df = pd.merge(
    df, 
    pres_results_national[['year','REP', 'DEM']],
    how='left', left_on='previous_year', right_on='year', suffixes=('','_previous_year_national'))

del df['year_previous_year_national']
del df['year_previous_year']

df['pvi'] = (df['REP'] * .5 + df['REP_previous_year'] * .5 ) - (df['REP_national'] *.5 + df['REP_previous_year_national'] * .5)
df['fte_pvi'] = (df['REP'] * .75 + df['REP_previous_year'] * .25 ) - (df['REP_national'] *.75 + df['REP_previous_year_national'] * .25)

df[df['year'] == 2016].head()
pvi_df = df

In [ ]:
%%R -i df

pres_results_by_state
ggplot(data=df,aes(x=year, y=pvi, group=state, alpha=.1,col = ifelse(state=='California','gray','red'))) + 
geom_line()

??

- https://fivethirtyeight.com/features/the-election-map-alone-doesnt-explain-the-2014-republican-wave/
- https://fivethirtyeight.com/features/the-2014-senate-elections-were-the-most-nationalized-in-decades/


- https://fivethirtyeight.com/features/demographics-not-hacking-explain-the-election-results/
- https://fivethirtyeight.com/features/its-not-all-about-clinton-the-midwest-was-getting-redder-before-2016/
- https://fivethirtyeight.com/features/demographics-arent-destiny-and-four-other-things-this-election-taught-me/
